In [1]:
#1. 加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
#2.定义超参数
BATCH_SIZE = 64 #每批处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #用GPU还是CPU
EPOCHS = 20 #训练数据集的轮次

In [3]:
#3.构建pipeline,对图像做处理
pipeline = transforms.Compose([
    transforms.ToTensor(), #将图片转化为tensor
    transforms.Normalize((0.1307,), (0.3081,)) #降低模型复杂度
])

In [4]:
#4. 下载，加载数据
from torch.utils.data import DataLoader

#下载数据集
train_set = datasets.MNIST("data",train = True, download = True, transform = pipeline )

test_set = datasets.MNIST("data",train = False, download = True, transform = pipeline )

#加载数据
train_loader = DataLoader(train_set, batch_size = BATCH_SIZE, shuffle = True)

test_loader = DataLoader(test_set, batch_size = BATCH_SIZE, shuffle = True)

In [5]:
##插入代码，显示MNIST中的图片
with open("./data/MNIST/raw/train-images-idx3-ubyte", "rb") as f :
    file = f.read()

In [6]:
image1= [int(str(item).encode('ascii'),16) for item in file[16 : 16+784] ]
print(image1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 24, 24, 24, 294, 310, 373, 38, 358, 597, 583, 295, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 54, 148, 340, 368, 595, 595, 595, 595, 595, 549, 370, 595, 578, 405, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 73, 568, 595, 595, 595, 595, 595, 595, 595, 595, 593, 147, 130, 130, 86, 57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 24, 537, 595, 595, 595, 595, 595, 408, 386, 583, 577, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 128, 342, 263, 595, 595, 517, 17, 0, 67, 340, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 1, 340, 595, 1

In [7]:
import cv2 
import numpy as np

image1_np = np.array(image1, dtype= np.uint8).reshape(28, 28, 1)

print(image1_np.shape)

(28, 28, 1)


In [8]:
cv2.imwrite("digit.jpg", image1_np)

True

In [9]:
#5. 构建网络模型
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,5) #1:灰度图片的通道  10:输出通道  5:卷积核
        self.conv2 = nn.Conv2d(10,20,3) #10:输入通道  20:输出通道  
        self.fc1 = nn.Linear(20*10*10, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        input_size = x.size(0) 
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        
        x = self.conv2(x)
        x = F.relu(x)
        
        x = x.view(input_size, -1) #拉平，-1 自动计算维度，20*10*10 =2000
                   
        x = self.fc1(x) #输入: batch*2000  输出:batch*500
        x = F.relu(x) #保持shape不变
                   
        x = self.fc2(x) #输入:batch*500  输出:batch*10
                   
        output = F.log_softmax(x, dim=1)  #计算
                   
        return output

In [10]:
#6. 定义优化器
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters())

In [11]:
#7. 定义训练方法
def train_model(model, device, train_loader, optimizer,epoch):
    # 模型训练
    model.train()
    for batch_index,(data , target) in enumerate(train_loader):
        #部署到device
        data, target = data.to(device), target.to(device)
        #梯度初始化为0
        optimizer.zero_grad()
        #训练后的结果
        output = model(data)
        #计算损失
        loss = F.cross_entropy(output, target)
        #找到概率值最大的下标
        pred = output.max(1, keepdim = True)
        #反向传播
        loss.backward()
        #参数优化
        optimizer.step()
        if batch_index % 3000 == 0:
            print('Train Epoch:{} \t  Loss:{:.6f}'.format(epoch, loss.item()))

In [12]:
#8. 定义测试方法
def test_model(model, device, test_loader):
    #模型验证
    model.eval()
    #正确率
    correct = 0.0
    #测试损失
    test_loss = 0.0
    with torch.no_grad():
        for data,target in test_loader:
            #部署到device上
            data, target = data.to(device), target.to(device)
            #测试数据
            output = model(data)
            #计算测试损失
            test_loss += F.cross_entropy(output, target).item()
            #找到概率值最大的下标
            pred = output.max(1,keepdim = True)[1]
            
            #累计正确的值
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print("Test - Average loss : {:.4f}, Accurracy ;{:.3f}\n".format(
                    test_loss, 100.0 * correct / len(test_loader.dataset)))

In [13]:
#9. 调用方法7/8
for epoch in range(1, EPOCHS + 1):
    train_model(model,DEVICE,train_loader,optimizer,epoch)
    test_model(model,DEVICE,test_loader)

Train Epoch:1 	  Loss:2.308477
Test - Average loss : 0.0008, Accurracy ;98.240

Train Epoch:2 	  Loss:0.031134
Test - Average loss : 0.0006, Accurracy ;98.730

Train Epoch:3 	  Loss:0.005985
Test - Average loss : 0.0007, Accurracy ;98.630

Train Epoch:4 	  Loss:0.018791
Test - Average loss : 0.0004, Accurracy ;99.160

Train Epoch:5 	  Loss:0.003338
Test - Average loss : 0.0004, Accurracy ;99.160

Train Epoch:6 	  Loss:0.000413
Test - Average loss : 0.0006, Accurracy ;98.930

Train Epoch:7 	  Loss:0.001474
Test - Average loss : 0.0007, Accurracy ;98.740

Train Epoch:8 	  Loss:0.009495
Test - Average loss : 0.0006, Accurracy ;98.960

Train Epoch:9 	  Loss:0.000017
Test - Average loss : 0.0007, Accurracy ;98.870

Train Epoch:10 	  Loss:0.000209
Test - Average loss : 0.0007, Accurracy ;98.940

Train Epoch:11 	  Loss:0.000025
Test - Average loss : 0.0006, Accurracy ;99.140

Train Epoch:12 	  Loss:0.000805
Test - Average loss : 0.0007, Accurracy ;98.930

Train Epoch:13 	  Loss:0.000212
Test 